In [11]:
import time

import pinocchio
import numpy as np
np.set_printoptions(precision=3, suppress=True)

from src import damped_IK

In [12]:
urdf_file = "panda_description/urdf/panda.urdf"

constraints = [{'frame': 'panda_joint7', 'type': 'position', 'weigth':1},
               {'frame': 'panda_joint7', 'type': 'orientation', 'weigth':1},
               ]

IK_solver = damped_IK(urdf_file, constraints)

q0 = pinocchio.randomConfiguration(IK_solver.model)

targets = [np.array([0.4, 0.2, 0.2]),
           np.array([[0,1,0], [1,0,0], [0,0,-1]])]

t0 = time.time()
q_path, success = IK_solver.run(q0, targets, return_path=True)
print(f"Solver successful: {success} in {1000*(time.time()-t0):.2f}ms with {len(q_path)} iterations")

print(q_path[-1])
print(IK_solver.data.oMi[7])

Solver successful: True in 42.45ms with 144 iterations
[ 1.968  1.584 -1.244 -2.379  1.818  1.784 -2.011  0.028  0.007]
  R =
-0.000438342            1 -0.000516766
    0.999999  0.000437691  -0.00126007
 -0.00125984 -0.000517318    -0.999999
  p = 0.399308 0.198309 0.201112



In [15]:
import pybullet as p
import time
import pybullet_data
physicsClient = p.connect(p.GUI)#or p.DIRECT for non-graphical version
robotSim = p.loadURDF(urdf_file, [0,0,0], p.getQuaternionFromEuler([0,0,0]))
p.resetDebugVisualizerCamera(cameraYaw = 90, cameraPitch= -20,cameraDistance = 1,cameraTargetPosition=[0.7,-0.2,0.7])

# Show target as red sphere
for constraint, target in zip(constraints, targets):
    if constraint['type'] == 'position':
        test_visual = p.createVisualShape(p.GEOM_SPHERE, radius=0.1, rgbaColor=[1,0,0,0.5])
        test_body = p.createMultiBody(baseMass=0, baseVisualShapeIndex=test_visual, basePosition = target)

# Replay robot trajectory to target     
for i, q in enumerate(q_path):
    for jointIndex in range(len(q)):
        p.resetJointState(robotSim, jointIndex, q[jointIndex])
        p.setJointMotorControl2(robotSim, jointIndex, p.POSITION_CONTROL,
                                targetPosition=q[jointIndex], force=10)
    if i==0: time.sleep(1)
    time.sleep(0.1)


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 11.0.1, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 20.3.5
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 20.3.5
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 11.0.1, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org


In [14]:
p.disconnect()

numActiveThreads = 0
stopping threads
destroy semaphore
semaphore destroyed
Thread with taskId 0 exiting
Thread TERMINATED
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
